In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pickle
import hickle
from tqdm import tqdm

In [2]:
# calculate the planet occurrence rate for a given stellar mass, semi-major axis, and planet mass (from Fulton et al. 2021)
# the 'powerlaw_index' and 'norm_constant' parameters allow for modifications to the stellar mass dependence
def planet_occurrence(M_star, a, C=350.0, beta=-0.86, a0=3.6, gamma=1.59, powerlaw_index=1.0, norm_constant=0.9):
    C_new = C*((M_star/norm_constant)**powerlaw_index)
    occurrence_rate = C_new * (a**beta) * (1.0 - np.exp(-(a/a0)**gamma))
    dlogm = np.log(6000) - np.log(30)
    occurrence_rate = occurrence_rate/(719.0*dlogm*0.63)

    return occurrence_rate # returns dNplanets / (dNstars * dlnm * dlna)

In [3]:
# function to calculate the number of planet detections, when provided the grid of searchable stars and the occurrence rates
# returns the overall number of detections and the number of detections around M dwarfs (stars with M < 0.6 M_sun)
def calculate_num_detections(nsearch, occ):
    nplanets = occ * nsearch
    num_detected, num_detected_Mdwarf = 0.0, 0.0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                num_detected += nplanets[i,j,k]
                if Mstar[k] < 0.6:
                    num_detected_Mdwarf += nplanets[i,j,k]
                    
    return num_detected, num_detected_Mdwarf

In [4]:
# load CLS posterior from Fulton et al. (2021)
f = open("data/Fulton21_posterior.pkl", "rb")
rand_Cs = pickle.load(f)
rand_Betas = pickle.load(f)
rand_a0s = pickle.load(f)
rand_gammas = pickle.load(f)

# Calculation for DR5

In [5]:
# setup 3D grid of a, mp, and Mstar
n = 50
alog = np.linspace(np.log(0.125), np.log(7.0), n) # log semi-major axis grid
mlog = np.linspace(np.log(0.3), np.log(13.0), n) # log planet mass grid
Mstar = np.linspace(0.1, 2.0, n) # stellar mass grid
dloga = alog[1] - alog[0]
dlogm = mlog[1] - mlog[0]
dMstar = Mstar[1] - Mstar[0]
a, m = np.exp(alog), np.exp(mlog)
amesh, mmesh, Mstarmesh = np.meshgrid(a, m, Mstar) # 3D grid

In [6]:
# load grid of searchable stars (only needs to be calculated once)
nsearch = hickle.load('Data/Nsearchable_grid_DR5.hkl')

In [7]:
# check number of searchable stars
tot_searchable = 0.0
for i in range(n):
    for j in range(n):
        for k in range(n):
            tot_searchable += nsearch[i,j,k]
tot_searchable

6854753.0886079725

In [8]:
# calculate planet detection numbers for different realizations of the Fulton et al. (2021) occurrence function
num_detections_F21, num_detections_Mdwarf_F21 = [], []
for i in tqdm(range(10_000)):
    # take random draw from Fulton et al. 2021 posterior
    rand_ind = np.random.choice(np.arange(len(rand_Cs)))
    rand_C, rand_Beta, rand_a0, rand_gamma = rand_Cs[rand_ind], rand_Betas[rand_ind], rand_a0s[rand_ind], rand_gammas[rand_ind]
    
    # calculate occurrence rate grid 
    occ = planet_occurrence(Mstarmesh, amesh, C=rand_C, beta=rand_Beta, a0=rand_a0, gamma=rand_gamma, powerlaw_index=1.0, norm_constant=0.9)
    
    # record number of detections
    num_detected, num_detected_Mdwarf = calculate_num_detections(nsearch, occ)
    num_detections_F21.append(num_detected)
    num_detections_Mdwarf_F21.append(num_detected_Mdwarf)

# convert to numpy arrays
num_detections_F21, num_detections_Mdwarf_F21 = np.array(num_detections_F21), np.array(num_detections_Mdwarf_F21)

100%|███████████████████████████████████████████████████████| 10000/10000 [02:49<00:00, 58.91it/s]


In [9]:
# uncertainty in total number due to draws from F21 posterior
np.median(num_detections_F21), np.std(num_detections_F21)

(124542.34098403034, 16072.54744425561)

In [10]:
# uncertainty in M-dwarf number due to draws from F21 posterior
np.median(num_detections_Mdwarf_F21), np.std(num_detections_Mdwarf_F21)

(29988.42425115219, 3786.7830729076713)

In [11]:
# calculate planet detection numbers for different versions of the stellar mass dependence of the occurrence rate
num_detections_SM, num_detections_Mdwarf_SM = [], []
for i in tqdm(range(10_000)):
    # draw random stellar mass dependence
    rand_powerlaw_index = np.random.uniform(0.0, 2.0)
    rand_norm_constant = np.random.uniform(0.8, 1.0)
    
    # calculate occurrence rate grid 
    occ = planet_occurrence(Mstarmesh, amesh, powerlaw_index=rand_powerlaw_index, norm_constant=rand_norm_constant)
    
    # record number of detections
    num_detected, num_detected_Mdwarf = calculate_num_detections(nsearch, occ)
    num_detections_SM.append(num_detected)
    num_detections_Mdwarf_SM.append(num_detected_Mdwarf)

# convert to numpy arrays
num_detections_SM, num_detections_Mdwarf_SM = np.array(num_detections_SM), np.array(num_detections_Mdwarf_SM)

100%|███████████████████████████████████████████████████████| 10000/10000 [02:56<00:00, 56.67it/s]


In [12]:
# uncertainty in total number due to stellar mass dependence
np.median(num_detections_SM), np.std(num_detections_SM)

(136298.96950636274, 18823.974467860306)

In [13]:
# uncertainty in M-dwarf number due to stellar mass dependence
np.median(num_detections_Mdwarf_SM), np.std(num_detections_Mdwarf_SM)

(30931.485500008584, 17535.621508546323)

In [14]:
# calculate planet detection numbers by combining both unknowns
num_detections_both, num_detections_Mdwarf_both = [], []
for i in tqdm(range(10_000)):
    # take random draw from Fulton et al. 2021 posterior
    rand_ind = np.random.choice(np.arange(len(rand_Cs)))
    rand_C, rand_Beta, rand_a0, rand_gamma = rand_Cs[rand_ind], rand_Betas[rand_ind], rand_a0s[rand_ind], rand_gammas[rand_ind]
    
    # draw random stellar mass dependence
    rand_powerlaw_index = np.random.uniform(0.0, 2.0)
    rand_norm_constant = np.random.uniform(0.8, 1.0)
    
    # calculate occurrence rate grid 
    occ = planet_occurrence(Mstarmesh, amesh, C=rand_C, beta=rand_Beta, a0=rand_a0, gamma=rand_gamma, powerlaw_index=rand_powerlaw_index, norm_constant=rand_norm_constant)
    
    # record number of detections
    num_detected, num_detected_Mdwarf = calculate_num_detections(nsearch, occ)
    num_detections_both.append(num_detected)
    num_detections_Mdwarf_both.append(num_detected_Mdwarf)

# convert to numpy arrays
num_detections_both, num_detections_Mdwarf_both = np.array(num_detections_both), np.array(num_detections_Mdwarf_both)

100%|███████████████████████████████████████████████████████| 10000/10000 [02:56<00:00, 56.80it/s]


In [15]:
# uncertainty in total number due to both sources
np.median(num_detections_both), np.std(num_detections_both)

(130113.93976411986, 25076.138355554438)

In [16]:
# uncertainty in M-dwarf number due to both sources
np.median(num_detections_Mdwarf_both), np.std(num_detections_Mdwarf_both)

(30146.193330667906, 18083.85411045946)

In [17]:
# relative uncertainty on total number
np.std(num_detections_both)/np.median(num_detections_both)

0.19272445674163974

In [18]:
# lower limit on M-dwarf detection number
np.percentile(num_detections_Mdwarf_both, 5.0)

13786.56777862529

# Calculation for DR4

In [19]:
# setup 3D grid of a, mp, and Mstar
n = 50
alog = np.linspace(np.log(0.125), np.log(7.0), n) # log semi-major axis grid
mlog = np.linspace(np.log(0.3), np.log(13.0), n) # log planet mass grid
Mstar = np.linspace(0.1, 2.0, n) # stellar mass grid
dloga = alog[1] - alog[0]
dlogm = mlog[1] - mlog[0]
dMstar = Mstar[1] - Mstar[0]
a, m = np.exp(alog), np.exp(mlog)
amesh, mmesh, Mstarmesh = np.meshgrid(a, m, Mstar) # 3D grid

In [20]:
# load grid of searchable stars (only needs to be calculated once)
nsearch = hickle.load('Data/Nsearchable_grid_DR4.hkl')

In [21]:
# check number of searchable stars
tot_searchable = 0.0
for i in range(n):
    for j in range(n):
        for k in range(n):
            tot_searchable += nsearch[i,j,k]
tot_searchable

1178479.0431254092

In [22]:
# calculate planet detection numbers for different realizations of the Fulton et al. (2021) occurrence function
num_detections_F21, num_detections_Mdwarf_F21 = [], []
for i in tqdm(range(10_000)):
    # take random draw from Fulton et al. 2021 posterior
    rand_ind = np.random.choice(np.arange(len(rand_Cs)))
    rand_C, rand_Beta, rand_a0, rand_gamma = rand_Cs[rand_ind], rand_Betas[rand_ind], rand_a0s[rand_ind], rand_gammas[rand_ind]
    
    # calculate occurrence rate grid 
    occ = planet_occurrence(Mstarmesh, amesh, C=rand_C, beta=rand_Beta, a0=rand_a0, gamma=rand_gamma, powerlaw_index=1.0, norm_constant=0.9)
    
    # record number of detections
    num_detected, num_detected_Mdwarf = calculate_num_detections(nsearch, occ)
    num_detections_F21.append(num_detected)
    num_detections_Mdwarf_F21.append(num_detected_Mdwarf)

# convert to numpy arrays
num_detections_F21, num_detections_Mdwarf_F21 = np.array(num_detections_F21), np.array(num_detections_Mdwarf_F21)

100%|███████████████████████████████████████████████████████| 10000/10000 [02:50<00:00, 58.49it/s]


In [23]:
# uncertainty in total number due to draws from F21 posterior
np.median(num_detections_F21), np.std(num_detections_F21)

(14438.127212031803, 1845.924544569323)

In [24]:
# uncertainty in M-dwarf number due to draws from F21 posterior
np.median(num_detections_Mdwarf_F21), np.std(num_detections_Mdwarf_F21)

(5549.865086431673, 700.4998035438553)

In [25]:
# calculate planet detection numbers for different versions of the stellar mass dependence of the occurrence rate
num_detections_SM, num_detections_Mdwarf_SM = [], []
for i in tqdm(range(10_000)):
    # draw random stellar mass dependence
    rand_powerlaw_index = np.random.uniform(0.0, 2.0)
    rand_norm_constant = np.random.uniform(0.8, 1.0)
    
    # calculate occurrence rate grid 
    occ = planet_occurrence(Mstarmesh, amesh, powerlaw_index=rand_powerlaw_index, norm_constant=rand_norm_constant)
    
    # record number of detections
    num_detected, num_detected_Mdwarf = calculate_num_detections(nsearch, occ)
    num_detections_SM.append(num_detected)
    num_detections_Mdwarf_SM.append(num_detected_Mdwarf)

# convert to numpy arrays
num_detections_SM, num_detections_Mdwarf_SM = np.array(num_detections_SM), np.array(num_detections_Mdwarf_SM)

100%|███████████████████████████████████████████████████████| 10000/10000 [02:58<00:00, 56.15it/s]


In [26]:
# uncertainty in total number due to stellar mass dependence
np.median(num_detections_SM), np.std(num_detections_SM)

(14858.427726946808, 3574.7088206549993)

In [27]:
# uncertainty in M-dwarf number due to stellar mass dependence
np.median(num_detections_Mdwarf_SM), np.std(num_detections_Mdwarf_SM)

(5617.459419232409, 3176.3447041131076)

In [28]:
# calculate planet detection numbers by combining both unknowns
num_detections_both, num_detections_Mdwarf_both = [], []
for i in tqdm(range(10_000)):
    # take random draw from Fulton et al. 2021 posterior
    rand_ind = np.random.choice(np.arange(len(rand_Cs)))
    rand_C, rand_Beta, rand_a0, rand_gamma = rand_Cs[rand_ind], rand_Betas[rand_ind], rand_a0s[rand_ind], rand_gammas[rand_ind]
    
    # draw random stellar mass dependence
    rand_powerlaw_index = np.random.uniform(0.0, 2.0)
    rand_norm_constant = np.random.uniform(0.8, 1.0)
    
    # calculate occurrence rate grid 
    occ = planet_occurrence(Mstarmesh, amesh, C=rand_C, beta=rand_Beta, a0=rand_a0, gamma=rand_gamma, powerlaw_index=rand_powerlaw_index, norm_constant=rand_norm_constant)
    
    # record number of detections
    num_detected, num_detected_Mdwarf = calculate_num_detections(nsearch, occ)
    num_detections_both.append(num_detected)
    num_detections_Mdwarf_both.append(num_detected_Mdwarf)

# convert to numpy arrays
num_detections_both, num_detections_Mdwarf_both = np.array(num_detections_both), np.array(num_detections_Mdwarf_both)

100%|███████████████████████████████████████████████████████| 10000/10000 [02:55<00:00, 57.12it/s]


In [29]:
# uncertainty in total number due to both sources
np.median(num_detections_both), np.std(num_detections_both)

(14961.653710220835, 4189.703417702862)

In [30]:
# uncertainty in M-dwarf number due to both sources
np.median(num_detections_Mdwarf_both), np.std(num_detections_Mdwarf_both)

(5578.056568233208, 3373.16629261486)

In [31]:
# relative uncertainty on total number
np.std(num_detections_both)/np.median(num_detections_both)

0.2800294338346253

In [32]:
# lower limit on M-dwarf detection number
np.percentile(num_detections_Mdwarf_both, 5.0)

2512.797212054531